In [1]:
import serial
from serial.tools import list_ports
import yaml
import cv2
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
import numpy as np
import os


In [2]:
# list the available serial ports

ports = serial.tools.list_ports.comports()

print([port.name for port in ports]) 

['ttyACM0', 'ttyAMA0']


In [3]:
# open the serial port
ser = serial.Serial('/dev/ttyACM0', 115200)  # baudrate = 115200

In [4]:
ser.is_open # did it really open? 

True

In [5]:
ser.write(b'G90\n') # explicitly set absolute positioning

4

In [8]:
ser.write(b'G1 X100 Y100 F6000\n') # go to a position!

19

In [ ]:
# now grab an image from the usb microscop & show it
videoCaptureObject = cv2.VideoCapture(0)
result = True
while(result):
    ret,frame = videoCaptureObject.read()
    # cv2.imwrite("test.jpg",frame)
    result = False
videoCaptureObject.release()
cv2.destroyAllWindows()
w,h,z = frame.shape
print(h)
plt.imshow(frame)
plt.title('frame capture')
cx, cy = plt.ginput(1)[0] # [number of clicks][single item list]
# plt.show()

print(cx)

In [6]:
# load in the calibration file
with open("./calibration/distance-calibration.yml", "r") as stream:
    try:
        dcal = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
# grab the x & y values
rx = dcal['z_50']['x']
ry = dcal['z_50']['y']
print(ry)

7.34


In [ ]:
# translate from pixels to real world
# the camera is rotated relative to the bed, so +x in cam space is -y on bed
# check this...
dy = np.interp(cx,[0,w],[-rx/2, rx/2])
dx = np.interp(cy, [0,h], [ry/2,-ry/2])

In [70]:
# now format to send over serial
dx = "{0:.2f}".format(dx)
dy = "{0:.2f}".format(dy)
print(dx)
print(dy)

-1.49
4.54


In [ ]:
## Send gcode
command = f'G1 X{dx} Y{dy} F6000\n'
ser.write(b'G91\n') # set relative positioning

4

In [72]:
ser.write(bytes(command, encoding='utf-8')) # send command

22

In [8]:
ser.write(b'G90\n') # restore absolute positioning

4

In [8]:
# image (a section of) the bed, stepping by the microscope viewport size
# save out each image
import os
for x in np.arange(130,150,rx):
    if not os.path.exists(f'./img_bed/{x}'):
        os.makedirs(f'./img_bed/{x}')
    for y in np.arange(150, 170, ry):
        cmd = f'G0 X{x} Y{y} F6000\n'
        ser.write(bytes(cmd, encoding='utf-8'))
        videoCaptureObject = cv2.VideoCapture(0)
        result = True
        while(result):
            ret,frame = videoCaptureObject.read()
            cv2.imwrite(f'./img_bed/{x}/{y}.jpg',frame)
            result = False
        videoCaptureObject.release()
        cv2.destroyAllWindows()

In [9]:
# now load in on horizontal row of images to concat them
images = {}
fnames = []
root = '/home/pi/duckweed/img_bed'
for subdir, dirs, files in os.walk(root):
    for d in dirs:
        if d not in images:
            images[d] = [] # initialize in the dict
        directory = os.path.join(subdir, d)
        for filename in sorted(os.listdir(directory)):
            img = cv2.imread(os.path.join(directory, filename))
            if img is not None:
                images[d].append(img)
        
# folder = '/home/pi/duckweed/img_bed/134.92'
# for filename in sorted(os.listdir(folder)):
#     fnames.append(os.path.join(folder,filename))
#     img = cv2.imread(os.path.join(folder,filename))
#     if img is not None:
#         images.append(img)

In [10]:
# concatenate each row
full_image = {}
for row in sorted(images):
    vconcat = cv2.vconcat(images[row])
    full_image[row] = vconcat
# cv2.imwrite('concat.jpg', concat)

In [11]:
# these are the concatenated rows
for row in full_image:
    cv2.imwrite(f'{row}.jpg', full_image[row])

In [12]:
# now hconcat those
image_stitch = []
for row in sorted(full_image, reverse=True):
    image_stitch.append(full_image[row])
hconcat = cv2.hconcat(image_stitch)
cv2.imwrite('full.jpg', hconcat)

True

In [13]:
# move to a position based on a click
# this time from the big image!
# now grab an image from the usb microscop & show it
bed = cv2.imread('full.jpg')
bed = np.rot90(bed, 3) # rotate 270 degrees
w,h,z = bed.shape
print(h)
plt.imshow(bed)
plt.title('frame capture')
pt = plt.ginput(1) # [number of clicks][single item list]
print(pt)

1440
[(506.5129870129871, 1570.4090909090908)]


In [14]:
cx, cy = pt[0]
xfactor = len(images) # this is x on the printer bed
yfactor = len(images[next(iter(images))]) # this is y on the printer bed
print(cx)
print(cy)

506.5129870129871
1570.4090909090908


In [15]:
# translate from pixels to real world
# depends on the imaged region size (i.e. how many pics were taken)
# the camera is rotated relative to the bed, so +x in cam space is -y on bed
# in each dimension, the bed position is the regionStart-camWidth/2 to regionStart + factor*camWidth + camWidth/2
xStart = 130
yStart = 150 ## these should be a variable depending on where you take pics
dy = np.interp(cx,[0,w],[yStart  + ry * yfactor - ry/2, yStart-ry/2])
dx = np.interp(cy, [0,h], [xStart + rx * xfactor - rx/2,xStart-rx/2])
# dy = np.interp(cx,[0,w],[yStart + rx * yfactor + rx/2, yStart-rx/2])
# dx = np.interp(cy, [0,h], [xStart + ry * xfactor + ry/2,xStart-ry/2])
print(dx)
print(dy)

125.04
162.54092918019484


3

In [16]:
ser.write(b'T2\n') # tool change
dx = "{0:.2f}".format(dx)
dy = "{0:.2f}".format(dy)
command = f"G1 X{dx} Y{dy} F6000\n"
ser.write(bytes(command, encoding='utf-8')) # send command

25

In [27]:
# Send tool down!
ser.write(b'G91\n') # Relative positioning
dz = -80.0
command = f"G1 Z{dz} F6000\n"
ser.write(bytes(command, encoding='utf-8')) # send command
ser.write(b'G90\n') # Restore Abolute positioning

4

In [26]:
# Bring tool down!
ser.write(b'G91\n') # Relative positioning
dz = 80.0
command = f"G1 Z{dz} F6000\n"
ser.write(bytes(command, encoding='utf-8')) # send command
ser.write(b'G90\n') # Restore Abolute positioning

4